# SPACE-X API

In [1]:
import requests
from pandas.io.json import json_normalize
import pandas as pd
import os,datetime
import numpy as np
date=datetime.datetime.now().date().isoformat()

#### Generating data storage folder

In [2]:
destfolder='./Data'
if not os.path.exists(destfolder):
    os.mkdir(destfolder)

___

## Retrieve Company info

In [3]:
api='https://api.spacexdata.com'
cinfo='/v3/info' # COMPANY INFO
req_SX_info=api+cinfo

In [4]:
response = requests.get(req_SX_info)
result = response.json()

In [5]:
info=json_normalize(result).T
info.columns=['Info']
info.to_csv(destfolder+"/Space_X_info.csv")

___

## Retrieve Upcoming Launchs Information

In [6]:
urlrockets='https://api.spacexdata.com/v3/launches/upcoming'

In [7]:
response = requests.get(urlrockets)
result = response.json()

In [8]:
upcoming_launches=json_normalize(result)

In [9]:
cols=(upcoming_launches.columns).unique().tolist()
upcoming_launches=upcoming_launches[cols]

In [10]:
mydicts_1=upcoming_launches['rocket.first_stage.cores']

data_1st_stage=pd.concat([pd.Series(d[0]) for d in mydicts_1], axis=1).T
data_1st_stage.columns=['rocket.first_stage.cores.'+item for item in data_1st_stage.columns]

mydicts_2=upcoming_launches['rocket.second_stage.payloads']

data_2nd_stage=pd.concat([pd.Series(d[0]) for d in mydicts_1], axis=1).T
data_2nd_stage.columns=['rocket.second_stage.payloads.'+item for item in data_2nd_stage.columns]

In [11]:
final_upcoming=pd.concat([upcoming_launches[upcoming_launches.columns.tolist()[:28]],data_1st_stage,
                          upcoming_launches[upcoming_launches.columns.tolist()[30:34]],data_2nd_stage,
                          upcoming_launches[upcoming_launches.columns.tolist()[36:]]],axis=1)

In [12]:
final_upcoming.to_csv(destfolder+"/"+date+"_Space_X_Upcoming_Launchs.csv")

___

## Retrieve Starman Information

In [13]:
urlstarman='https://api.spacexdata.com/v3/roadster'

In [14]:
response = requests.get(urlstarman)
result = response.json()

In [15]:
starman=json_normalize(result)
starman=starman.drop_duplicates(['period_days'])
if not os.path.exists('./Data/Space_X_Starman.csv'):
    starman.to_csv('./Data/Space_X_Starman.csv',index=False)
else:
    data=pd.read_csv('./Data/Space_X_Starman.csv')
    data.columns=starman.columns.tolist()
    datad=pd.concat([starman,data],axis=0,sort=True)
    datad=datad.drop_duplicates(['period_days'])
    datad.to_csv('./Data/Space_X_Starman.csv',index=False)
    

### With crontab to make a request each hour
These results will be updated every 10 mins, so Im going to create a script .py to automatize it with crontab

____